In [9]:

!pip install -q fastapi uvicorn nest-asyncio pyngrok gradio scikit-learn pandas seaborn matplotlib


In [10]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Đọc dữ liệu
df = pd.read_csv("/content/Iris.csv")
df.drop('Id', axis=1, inplace=True)

# Encode nhãn
le = LabelEncoder()
df["Species"] = le.fit_transform(df["Species"])

# Tách đặc trưng và nhãn
X = df.drop("Species", axis=1)
y = df["Species"]

# Chuẩn hóa
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình tốt nhất
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Lưu mô hình
joblib.dump(model, "iris_best_model.pkl")
joblib.dump(scaler, "iris_scaler.pkl")
joblib.dump(le, "iris_label_encoder.pkl")


['iris_label_encoder.pkl']

In [11]:

%%writefile iris_api.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

app = FastAPI()

model = joblib.load("iris_best_model.pkl")
scaler = joblib.load("iris_scaler.pkl")
label_encoder = joblib.load("iris_label_encoder.pkl")

class IrisInput(BaseModel):
    SepalLengthCm: float
    SepalWidthCm: float
    PetalLengthCm: float
    PetalWidthCm: float

@app.post("/predict")
def predict_species(data: IrisInput):
    features = np.array([
        [data.SepalLengthCm, data.SepalWidthCm, data.PetalLengthCm, data.PetalWidthCm]
    ])
    scaled = scaler.transform(features)
    prediction = model.predict(scaled)
    label = label_encoder.inverse_transform(prediction)[0]
    return {"prediction": label}


Overwriting iris_api.py


In [ ]:

from pyngrok import ngrok
import nest_asyncio
import uvicorn

ngrok.set_auth_token("2zDalgx2Q0FAhEpPHeNVPGxzIRM_xsTkJgTi9mZv5EA5eSrq")
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)

# Lưu URL cho Gradio
api_url = public_url.public_url + "/predict"

# Chạy FastAPI server (cell này sẽ giữ kernel hoạt động)
uvicorn.run("iris_api:app", host="0.0.0.0", port=8000)


🔗 Public URL: NgrokTunnel: "https://f2d1-34-171-35-97.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [335]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     172.31.38.1:0 - "GET /static/fonts/IBMPlexMono/IBMPlexMono-Regular.woff2 HTTP/1.1" 200 OK
INFO:     172.31.38.1:0 - "POST /gradio_api/queue/join HTTP/1.1" 200 OK
INFO:     34.171.35.97:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     172.31.38.1:0 - "GET /gradio_api/queue/data?session_hash=ouxzl5m9i1i HTTP/1.1" 200 OK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:

import gradio as gr
import requests

def predict_flower(sepal_length, sepal_width, petal_length, petal_width):
    data = {
        "SepalLengthCm": sepal_length,
        "SepalWidthCm": sepal_width,
        "PetalLengthCm": petal_length,
        "PetalWidthCm": petal_width
    }
    response = requests.post(api_url, json=data)
    if response.status_code == 200:
        return response.json()["prediction"]
    else:
        return "Lỗi dự đoán!"

demo = gr.Interface(
    fn=predict_flower,
    inputs=[
        gr.Number(label="Sepal Length (cm)"),
        gr.Number(label="Sepal Width (cm)"),
        gr.Number(label="Petal Length (cm)"),
        gr.Number(label="Petal Width (cm)")
    ],
    outputs=gr.Textbox(label="Kết quả dự đoán"),
    title="🌸 Dự đoán Loài Hoa Iris"
)

demo.launch(share=True)
